# Generates gCloud CLI Commands Based On User's Input
You can see a UI version of this [here](https://cligenerator.xyz/).
Supporting Github repo is [here](https://github.com/UriKatsirPrivate/CLI-Generator).

## Prerequisites

1.   GCP Project
2.   [Enable](https://cloud.google.com/apis/docs/getting-started#enabling_apis) Vertex AI API
3.   IAM User with *Vertex AI User* permissions
  *   Uae this IAM user to login to this colab and to authenticate to Google Cloud in the step below

## Setup

### Install Libraries

In [ ]:
!pip -q install google-cloud-aiplatform==1.36.2
!pip -q install langchain==0.0.329

### ⚠️ Restart the session:

Runtime--> Restart runtime or run the 2 rows below

In [ ]:
import os
os.kill(os.getpid(), 9)

### 🔐 Authenticate to Google Cloud

Use the same user you configured during the Prerequisites step above.

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

## Building Blocks

### Input Variables

In [ ]:
# @title { run: "auto" }
project_id = "*** REQUIRED ***" # @param {type:"string"}
region = "us-central1" # @param {type:"string"}
model_name = "text-bison-32k" # @param ["text-bison", "text-bison-32k", "code-bison", "code-bison-32k"]
max_tokens = 8192 # @param {type:"slider", min:1, max:8192, step:1}
temperature = "0.1" # @param {type:"string"}
top_p = "0.8" # @param {type:"string"}
top_k = "40" # @param {type:"string"}


if not ('32k' in model_name) and max_tokens>1024:
  raise RuntimeError(f'{max_tokens} output tokens is not a valid value for model {model_name}')


### Enable APIs

In [ ]:
!gcloud services enable aiplatform.googleapis.com --project {project_id} --async

## Helper Functions

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate,
                                    HumanMessagePromptTemplate,
                                    SystemMessagePromptTemplate)
from langchain.llms import VertexAI

In [ ]:
def initialize_llm(project_id,region,model_name,max_output_tokens,temperature,top_p,top_k):

    # Initialize VertexAI and set up the LLM
    return VertexAI(
        project=project_id,
        location=region,
        model_name=model_name,
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        verbose=True,
    )

In [ ]:
def gcpCliCommandGenerator(user_input):

    llm = initialize_llm(project_id,region,model_name,max_tokens,temperature,top_p,top_k)

    system_template = """You are a virtual assistant capable of generating the corresponding Google Cloud Platform (GCP) command-line interface (CLI) command based on the user's input."""
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
    human_template = """The user's input is: '{user_input}'. Please generate the corresponding GCP CLI command."""
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt, human_message_prompt]
    )

    chain = LLMChain(llm=llm, prompt=chat_prompt)
    result = chain.run(user_input=user_input)
    return result # returns string

## Execute the Service

In [ ]:
user_input = "create vpc" # @param {type:"string"}
gcp_command = gcpCliCommandGenerator(user_input)

In [ ]:
print(gcp_command)